EXPLORING ASTRONAUTS ACTIVITIES IN SQL

Processed and analyzed a database including data on astronaut Extra-Vehicular Activities(EVAs) using string cleaning.

Exploring the data and looking for potential problems with the text-filled columns were all parts of the process. I then cleaned up the data so I could learn more about the different kinds of EVAs and their participants!

This provides answers to queries.
1. Frequently occurring EVAs with CASE expressions
2. The amount of data retrieved and processed using CTEs, Regular expressions, and SUBSTRING
3. The eVA veteran astronaut employed CTEs, SPLIT PART, and UNION ALL
4. The cumulative time spent using window functions

Key Learnings

-Identifying potential issues with text data using SQL
-Reformating and clean up messy text data
-Basics of regular expressions and how they can be used to categorize data

In [11]:
-- Start writing your SQL query here 

SELECT *
FROM evas

,date,country,vehicle,duration,crew,purpose,year,program
0,1965-06-03,USA,Gemini IV,36,Ed White,First U.S. EVA. Used HHMU and took photos. G...,1965,Gemini
1,1966-03-16,USA,Gemini VIII,0,David Scott,HHMU EVA cancelled before starting by stuck on...,1966,Gemini
2,1966-06-05,USA,Gemini IX-A,127,Eugene Cernan,"Inadequate restraints, stiff 25ft umbilical an...",1966,Gemini
3,1966-07-19,USA,Gemini X,50,Mike Collins,Standup EVA. UV photos of stars. Ended by ey...,1966,Gemini
4,1966-07-20,USA,Gemini X,39,Mike Collins,Retrieved MMOD experiment from docked Agena. ...,1966,Gemini
...,...,...,...,...,...,...,...,...
370,2012-08-20,Russia,ISS Incr-32,351,"Gennady Padalka, Yuri Malenchenko",1 hr late start due to airlock valve. Relocat...,2012,ISS
371,2013-04-19,Russia,ISS Incr-35,398,"Pavel Vinogradov, Roman Romanenko","Installed plasma experiment/cables/probes, rep...",2013,ISS
372,2013-06-24,Russia,ISS Incr-36,394,"Fyodor Yurchikhin, Alexander Misurkin",Power cable clamps installed and Kurs tested i...,2013,ISS
373,2013-08-16,Russia,ISS Incr-36,449,"Fyodor Yurchikhin, Alexander Misurkin","Â Installed VINOSLIVOST experiment on MRM2, 2...",2013,ISS


In [12]:
SELECT purpose
FROM evas

,purpose
0,First U.S. EVA. Used HHMU and took photos. G...
1,HHMU EVA cancelled before starting by stuck on...
2,"Inadequate restraints, stiff 25ft umbilical an..."
3,Standup EVA. UV photos of stars. Ended by ey...
4,Retrieved MMOD experiment from docked Agena. ...
...,...
370,1 hr late start due to airlock valve. Relocat...
371,"Installed plasma experiment/cables/probes, rep..."
372,Power cable clamps installed and Kurs tested i...
373,"Â Installed VINOSLIVOST experiment on MRM2, 2..."


What are the common EVAS?
Use Case expressions..

In [13]:
SELECT 
	purpose,
    CASE WHEN purpose ILIKE '%photos%' THEN 1 ELSE 0 END AS Photography,
    CASE WHEN purpose ILIKE '%collection%' THEN 1 ELSE 0 END AS Collection,
    CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE 'assembl%'THEN 1 ELSE 0 END AS Installation,
    CASE WHEN purpose ILIKE '%replace%' OR purpose ILIKE '%fix%' OR purpose ILIKE '%repair%' OR purpose ILIKE '%servic%'THEN 1 ELSE 0 END AS Repair
    
FROM evas

,purpose,photography,collection,installation,repair
0,First U.S. EVA. Used HHMU and took photos. G...,1,0,0,0
1,HHMU EVA cancelled before starting by stuck on...,0,0,0,0
2,"Inadequate restraints, stiff 25ft umbilical an...",0,0,0,0
3,Standup EVA. UV photos of stars. Ended by ey...,1,0,0,0
4,Retrieved MMOD experiment from docked Agena. ...,0,0,0,0
...,...,...,...,...,...
370,1 hr late start due to airlock valve. Relocat...,0,0,1,1
371,"Installed plasma experiment/cables/probes, rep...",0,0,1,1
372,Power cable clamps installed and Kurs tested i...,0,0,1,1
373,"Â Installed VINOSLIVOST experiment on MRM2, 2...",0,0,1,0


In [17]:
With purposes AS (
   SELECT 
        purpose,
        CASE WHEN purpose ILIKE '%photos%' THEN 1 ELSE 0 END AS Photography,
        CASE WHEN purpose ILIKE '%collection%' THEN 1 ELSE 0 END AS Collection,
        CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE 'assembl%'THEN 1 ELSE 0 END AS Installation,
        CASE WHEN purpose ILIKE '%replace%' OR purpose ILIKE '%fix%' OR purpose ILIKE '%repair%' OR purpose ILIKE '%servic%'THEN 1 ELSE 0 END AS Repair

    FROM evas
    )
    
SELECT 
	SUM(Photography) AS count,
    'photography' AS type
FROM purposes
UNION
SELECT 
	SUM(Collection) AS count,
    'collection' AS type
FROM purposes
UNION
SELECT 
	SUM(Installation) AS count,
    'installation' AS type
FROM purposes
UNION
SELECT 
	SUM(Repair) AS count,
    'repair' AS type
FROM purposes
ORDER BY count DESC


,count,type
0,184,installation
1,129,repair
2,13,photography
3,1,collection


In [18]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,count,type
0,184,installation
1,129,repair
2,13,photography
3,1,collection


How much material extracted from EVA?

In [20]:
SELECT 
	purpose
FROM evas
WHERE purpose ILIKE '%geologic%' OR purpose ILIKE '%rock%'

,purpose
0,First to walk on the moon. Some trouble getti...
1,Collected 75.6 lb of geologic material. ALSEP...
2,Collected 94.4 lb of geologic material. ALSEP...
3,Collected 169 lb of geologic material. ALSEP ...
4,Collected 208 lb of rock/dust (41lb this day)....
5,Collected 82 lb of rock/dust. Drove rover 11.5 km
6,Collected 90 lb of rock/dust. Drove rover 27....
7,Collected 243 lb of geologic material. ALSEP ...


In [3]:
SELECT 
	purpose,
    SUBSTRING(purpose , '(\d+\.?\d*) lb of ((rock|geologic))') AS weight
FROM evas

,purpose,weight
0,First U.S. EVA. Used HHMU and took photos. G...,NaN
1,HHMU EVA cancelled before starting by stuck on...,NaN
2,"Inadequate restraints, stiff 25ft umbilical an...",NaN
3,Standup EVA. UV photos of stars. Ended by ey...,NaN
4,Retrieved MMOD experiment from docked Agena. ...,NaN
...,...,...
370,1 hr late start due to airlock valve. Relocat...,NaN
371,"Installed plasma experiment/cables/probes, rep...",NaN
372,Power cable clamps installed and Kurs tested i...,NaN
373,"Â Installed VINOSLIVOST experiment on MRM2, 2...",NaN


CTE to calaculate total amount

In [4]:
	WITH weights AS (
        SELECT 
            purpose,
            SUBSTRING(purpose , '(\d+\.?\d*) lb of ((rock|geologic))') AS weight
        FROM evas
    )
    SELECT SUM(weight :: numeric)
    FROM weights

,sum
0,1008.3


Which Astronaut has most time in EVAs?

In [5]:
SELECT 
	crew,
    SPLIT_PART(crew , ',' , 4)
FROM evas
WHERE SPLIT_PART(crew , ',' , 4) != ''

,crew,split_part


In [7]:
WITH astronauts_split AS(
	SELECT
    	crew,
    	SPLIT_PART(crew, ',',1) AS first_astronaut,
    	SPLIT_PART(crew, ',',2) AS second_astronaut,
    	SPLIT_PART(crew, ',',3) AS third_astronaut,
    	duration
	FROM evas
)
SELECT 
	*
FROM astronauts_split

,crew,first_astronaut,second_astronaut,third_astronaut,duration
0,Ed White,Ed White,,,36
1,David Scott,David Scott,,,0
2,Eugene Cernan,Eugene Cernan,,,127
3,Mike Collins,Mike Collins,,,50
4,Mike Collins,Mike Collins,,,39
...,...,...,...,...,...
370,"Gennady Padalka, Yuri Malenchenko",Gennady Padalka,Yuri Malenchenko,,351
371,"Pavel Vinogradov, Roman Romanenko",Pavel Vinogradov,Roman Romanenko,,398
372,"Fyodor Yurchikhin, Alexander Misurkin",Fyodor Yurchikhin,Alexander Misurkin,,394
373,"Fyodor Yurchikhin, Alexander Misurkin",Fyodor Yurchikhin,Alexander Misurkin,,449


In [9]:
WITH astronauts_split AS(
	SELECT
    	crew,
    	SPLIT_PART(crew, ',',1) AS first_astronaut,
    	SPLIT_PART(crew, ',',2) AS second_astronaut,
    	SPLIT_PART(crew, ',',3) AS third_astronaut,
    	duration
	FROM evas
),
astronaut_duration AS (
	SELECT 
    	first_astronaut AS astronaut,
    	duration
    FROM astronauts_split
    WHERE first_astronaut != ''
    UNION ALL
    SELECT 
    	second_astronaut AS astronaut,
    	duration
    FROM astronauts_split
    WHERE second_astronaut != ''
    UNION ALL
    SELECT 
    	third_astronaut AS astronaut,
    	duration
    FROM astronauts_split
    WHERE third_astronaut != ''
)

SELECT 
	astronaut,
    SUM(duration) AS total_duration
FROM astronaut_duration
GROUP BY astronaut
ORDER BY total_duration DESC
LIMIT 10

,astronaut,total_duration
0,Jerry Ross,3501
1,Anatoly Solovyev,3086
2,Scott Parazynski,2825
3,Nikola Budarin,2672
4,John Grunsfeld,2527
5,Mike Lopez-Alegria,2501
6,Mike Fincke,2472
7,Dan Tani,2351
8,Victor Afanasyev,2314
9,Rick Mastracchio,2311


In [10]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,astronaut,total_duration
0,Jerry Ross,3501
1,Anatoly Solovyev,3086
2,Scott Parazynski,2825
3,Nikola Budarin,2672
4,John Grunsfeld,2527
5,Mike Lopez-Alegria,2501
6,Mike Fincke,2472
7,Dan Tani,2351
8,Victor Afanasyev,2314
9,Rick Mastracchio,2311


Cumulative Time spent in the EVA over time (Window Functions)

In [17]:
SELECT
	TO_DATE(year::text,'YYYY') AS year,
    program,
    duration,
    SUM(duration)OVER(PARTITION BY program ORDER BY year) AS cumulative_duration
FROM(
   SELECT 
        year,
        program,
        SUM(duration) AS duration
    FROM evas
    GROUP BY year, program
    ORDER BY year, program
    ) AS sub
ORDER BY year, program

,year,program,duration,cumulative_duration
0,1965-01-01 00:00:00+00:00,Gemini,36,36
1,1965-01-01 00:00:00+00:00,Voskhod,12,12
2,1966-01-01 00:00:00+00:00,Gemini,720,756
3,1969-01-01 00:00:00+00:00,Apollo,707,707
4,1969-01-01 00:00:00+00:00,Soyuz,37,37
...,...,...,...,...
63,2010-01-01 00:00:00+00:00,Space Shuttle,3591,61140
64,2011-01-01 00:00:00+00:00,ISS,1388,17901
65,2011-01-01 00:00:00+00:00,Space Shuttle,2492,63632
66,2012-01-01 00:00:00+00:00,ISS,2009,19910


In [18]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,year,program,duration,cumulative_duration
0,1965-01-01 00:00:00+00:00,Gemini,36,36
1,1965-01-01 00:00:00+00:00,Voskhod,12,12
2,1966-01-01 00:00:00+00:00,Gemini,720,756
3,1969-01-01 00:00:00+00:00,Apollo,707,707
4,1969-01-01 00:00:00+00:00,Soyuz,37,37
...,...,...,...,...
63,2010-01-01 00:00:00+00:00,Space Shuttle,3591,61140
64,2011-01-01 00:00:00+00:00,ISS,1388,17901
65,2011-01-01 00:00:00+00:00,Space Shuttle,2492,63632
66,2012-01-01 00:00:00+00:00,ISS,2009,19910
